##### 问题
一些无聊的幼稚黑客在你的网站页面表单中输入文本“pýthöñ”，然后你想将这些字符清理掉。

##### 解决方案
文本清理问题会涉及到包括**文本解析与数据处理**等一系列问题。在非常简单的情形下，你可能会选择使用字符串函数将文本转为比标准格式。使用str.replace()或者re.sub()的简单替换操作能删除或者改变指定的字符序列。同样可以使用unicodedata模块将unicode文本标准化。  
然后，有时你可能还想在清理啊哦做上更进一步。比如，你可能想消除整个区间上的字符或者去除变音符。为了这样做，你可以使用str.translate()方法。

In [1]:
s = 'pýthöñ\fis\tawesome\r\n'
s

'pýthöñ\x0cis\tawesome\r\n'

第一步是清理空白字符。为了这样做，先创建一个小的转换表格然后使用translate()方法：

In [2]:
remap = {
    ord('\t'): ' ',
    ord('\f'): ' ',
    ord('\r'): None
}
a = s.translate(remap)
a

'pýthöñ is awesome\n'

正如你所见，空白字符\t和\f被重新映射到一个空格。回车字符r直接被删除。  
你可以以这个表格为基础进一步构建更大的表格。比如，让我们删除所有的和音符：

In [4]:
import unicodedata
import sys

cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))
b = unicodedata.normalize('NFD', a)
b

'pýthöñ is awesome\n'

In [5]:
b.translate(cmb_chrs)

'python is awesome\n'

上面例子中，通过使用dict.fromkeys()方法构造一个字典，每个Unicode和音符作为键，对应的值全部为None。  
然后使用unicodedata.normalize()将原始输入标准化为分解形式字符。然后在调用translate函数删除所有重音符。同样的技术也可以被用来删除其他类型的字符。  
作为另一个例子，这里构造一个将所有Unicode数字字符映射到对应的ASCII字符上的表格：

In [7]:
digitmap = {c: ord('0') + unicodedata.digit(chr(c)) for c in range(sys.maxunicode) if unicodedata.category(chr(c)) == 'Nd'}
len(digitmap)

610

In [8]:
x = '\u0661\u0662\u0663'
x.translate(digitmap)

'123'

另一种清理文本的技术涉及到了I/O解码与编码函数。这里的思路是先对文本做一些初步的清理，然后在结合encode()或decode()操作来清除或修改它。比如：

In [9]:
a

'pýthöñ is awesome\n'

In [10]:
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

'python is awesome\n'

这里的标准化将原来的文本分解分解为单独的和音符。接下来的ASCII编码/解码只是简单的一下子丢弃那些字符。当然，这种方法仅仅指在最后的目标就是获取到文本对应ACSII表时生效。

##### 讨论
文本字符清理一个最主要的问题应该是运行的性能。一般来讲，代码越简单运行越快。对比简单的替换操作，str.replace()方法是最快的，甚至在你需啊哟多次调用的时候。比如，为了清理空白字符，你可以这样做、

In [14]:
def clean_spaces(s):
    s = s.replace('\r', '')
    s = s.replace('\t', ' ')
    s = s.replace('\f', ' ')
    return s

如果你去测试，你会发现这种方式会比使用translate()或正则表达式要快很多。  
另一方面，如果你需要执行任何复杂字符对字符的重新映射或者删除操作的话，translate()方法会非常的快。 